In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
cd /content/drive/My Drive/flipchal

/content/drive/My Drive/flipchal


In [0]:
ls

id_to_box   id_to_data2  id_to_size3  test1.csv     test.gsheet
id_to_box2  id_to_data3  id_to_size4  test1.gsheet  test_set/
id_to_box3  id_to_data4  model2.h5    test2.csv     training.csv
id_to_box4  id_to_size   model.h5     test3.csv     train_set/
id_to_data  id_to_size2  p1/          test.csv      Untitled.ipynb


In [0]:
import numpy as np
import pandas as pd
from PIL import Image
import pickle
import random

In [0]:
def Normalize(image,mean,std):
    for channel in range(3):
        image[:,:,channel]=(image[:,:,channel]-mean[channel])/std[channel]
    return image

In [0]:
df = pd.read_csv('./test.csv')

In [0]:
df.head()

,image_name,x1,x2,y1,y2
0,1474723840903DSC08089.png,NaN,NaN,NaN,NaN
1,1473231475010DeeplearnS11276.png,NaN,NaN,NaN,NaN
2,JPEG_20161205_135307_1000155917326.png,NaN,NaN,NaN,NaN
3,JPEG_20160711_123440_1000518778437.png,NaN,NaN,NaN,NaN
4,JPEG_20160803_115329_100034020722.png,NaN,NaN,NaN,NaN


In [0]:
def getImgArray(id):
  path = df.iloc[id]['image_name']
  image=Image.open("./test_set/"+path).convert('RGB')
  image=image.resize((224,224))
  image=np.array(image,dtype=np.float32)
  image=image/255
  image=Normalize(image,[0.485,0.456,0.406],[0.229,0.224,0.225])
  inc_dim = {}
  inc_dim[0] = image
  inc_dim=np.array(list(inc_dim.values()))
  return inc_dim

In [0]:
def my_metric(labels,predictions):
    threshhold=0.75
    x=predictions[:,0]*224
    x=tf.maximum(tf.minimum(x,224.0),0.0)
    y=predictions[:,1]*224
    y=tf.maximum(tf.minimum(y,224.0),0.0)
    width=predictions[:,2]*224
    width=tf.maximum(tf.minimum(width,224.0),0.0)
    height=predictions[:,3]*224
    height=tf.maximum(tf.minimum(height,224.0),0.0)
    label_x=labels[:,0]
    label_y=labels[:,1]
    label_width=labels[:,2]
    label_height=labels[:,3]
    a1=tf.multiply(width,height)
    a2=tf.multiply(label_width,label_height)
    x1=tf.maximum(x,label_x)
    y1=tf.maximum(y,label_y)
    x2=tf.minimum(x+width,label_x+label_width)
    y2=tf.minimum(y+height,label_y+label_height)
    IoU=tf.abs(tf.multiply((x1-x2),(y1-y2)))/(a1+a2-tf.abs(tf.multiply((x1-x2),(y1-y2))))
    condition=tf.less(threshhold,IoU)
    sum=tf.where(condition,tf.ones(tf.shape(condition)),tf.zeros(tf.shape(condition)))
    return tf.reduce_mean(sum)

In [0]:
def smooth_l1_loss(true_box,pred_box):
    loss=0.0
    for i in range(4):
        residual=tf.abs(true_box[:,i]-pred_box[:,i]*224)
        condition=tf.less(residual,1.0)
        small_res=0.5*tf.square(residual)
        large_res=residual-0.5
        loss=loss+tf.where(condition,small_res,large_res)
    return tf.reduce_mean(loss)

In [0]:
import tensorflow as tf
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, MaxPooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model,load_model
from keras.callbacks import ModelCheckpoint,LearningRateScheduler,ReduceLROnPlateau

Using TensorFlow backend.


In [0]:
model=keras.models.load_model("./model2.h5", custom_objects={'smooth_l1_loss':smooth_l1_loss, 'my_metric':my_metric})

In [0]:
result=model.predict(img)

In [0]:
prediction = result[0]

In [0]:
prediction

In [0]:
print(str(prediction[0]*640)+' '+str(prediction[1]*480)+' '+str((prediction[2]+prediction[0])*640)+' '+str((prediction[3]+prediction[1])*480)+' ')

NameError: ignored

In [0]:
df.shape

(12815, 5)

In [0]:
finAns = {}

In [0]:
for i in range(12815):
  img = getImgArray(i)
  result = model.predict(img)
  prediction = result[0]
  prediction[0] = prediction[0]*640
  prediction[1] = prediction[1]*480
  prediction[2] = prediction[0]+prediction[2]*640
  prediction[3] = prediction[1]+prediction[3]*480
  finAns[i] = prediction
  if i%500 == 0:
    print("Step: "+str(i))

Step: 0
Step: 500
Step: 1000
Step: 1500
Step: 2000
Step: 2500
Step: 3000
Step: 3500
Step: 4000
Step: 4500
Step: 5000
Step: 5500
Step: 6000
Step: 6500
Step: 7000
Step: 7500
Step: 8000
Step: 8500
Step: 9000
Step: 9500
Step: 10000
Step: 10500
Step: 11000
Step: 11500
Step: 12000
Step: 12500


In [0]:
finAns

array([[240.16718 ,  81.38599 , 432.7739  , 394.47372 ],
       [ 77.35289 , 141.3264  , 568.66516 , 347.89807 ],
       [134.26187 ,  56.75032 , 489.63034 , 423.7465  ],
       ...,
       [ 33.4302  ,  26.419931, 455.29144 , 449.97208 ],
       [219.36893 , 134.60803 , 532.5951  , 352.115   ],
       [ 20.403927, 112.13265 , 595.7431  , 371.5594  ]], dtype=float32)

In [0]:
finAns = np.array(list(finAns.values()))

AttributeError: ignored

In [0]:
finAns

array([[240.16718 ,  81.38599 , 432.7739  , 394.47372 ],
       [ 77.35289 , 141.3264  , 568.66516 , 347.89807 ],
       [134.26187 ,  56.75032 , 489.63034 , 423.7465  ],
       ...,
       [ 33.4302  ,  26.419931, 455.29144 , 449.97208 ],
       [219.36893 , 134.60803 , 532.5951  , 352.115   ],
       [ 20.403927, 112.13265 , 595.7431  , 371.5594  ]], dtype=float32)

In [0]:
for i in range(12815):
  for j in range(4):
    if(finAns[i][j]<0):
      finAns[i][j] = 0
  if(finAns[i][2]>640):
    finAns[i][2] = 640
  if(finAns[i][3]>480):
    finAns[i][3] = 480

In [0]:
finAns[557]

array([ 21.96848,   0.     , 640.     , 448.35846], dtype=float32)

In [0]:
dataFrame = pd.DataFrame({'x1':finAns[:,0],'y1':finAns[:,1], 'x2':finAns[:,2], 'y2':finAns[:,3]})

In [0]:
dataFrame.head()

,x1,x2,y1,y2
0,240.167175,432.773895,81.385986,394.473724
1,77.352890,568.665161,141.326401,347.898071
2,134.261871,489.630341,56.750320,423.746490
3,209.363770,445.125519,88.439995,413.595367
4,132.681076,484.715393,40.573833,442.267853


In [0]:
names = df['image_name']

In [0]:
mdf = pd.DataFrame({'image_name':names})

In [0]:
mdf.head()

,image_name
0,1474723840903DSC08089.png
1,1473231475010DeeplearnS11276.png
2,JPEG_20161205_135307_1000155917326.png
3,JPEG_20160711_123440_1000518778437.png
4,JPEG_20160803_115329_100034020722.png


In [0]:
df_all_cols = pd.concat([mdf,dataFrame ], axis = 1)
df_all_cols

,image_name,x1,x2,y1,y2
0,1474723840903DSC08089.png,240.167175,432.773895,81.385986,394.473724
1,1473231475010DeeplearnS11276.png,77.352890,568.665161,141.326401,347.898071
2,JPEG_20161205_135307_1000155917326.png,134.261871,489.630341,56.750320,423.746490
3,JPEG_20160711_123440_1000518778437.png,209.363770,445.125519,88.439995,413.595367
4,JPEG_20160803_115329_100034020722.png,132.681076,484.715393,40.573833,442.267853
5,147444974116511473239803010-Mast--Harbour-Men-...,176.592804,356.167053,5.840404,467.596222
6,JPEG_20160622_110649_1000527459853.png,131.923569,507.590149,73.030663,360.061127
7,JPEG_20160823_120737_1000784898268.png,84.360130,426.641479,118.394180,372.035767
8,1480965956645IMG_3094.png,197.808670,423.739960,165.883423,386.336334
9,14732348976417a565e40d545452688130062b267a2d2.png,26.311104,583.110107,0.000000,480.000000


In [0]:
df_all_cols.to_csv('./test4.csv', index = False)